In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeWorkingHRDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "fitbit/17-9-24/takeout-20240917T195619Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)


In [3]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")


read in 16052785 rows from 19 files


In [4]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
else:
    hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]

2024-09-14


In [5]:
#takes like 40 minutes to make 15.8 million rows
import json

columnNames = ["sampleDT", "confidence", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        samplesList.append([pd.to_datetime(row["dateTime"] + "+0000").tz_convert("US/Arizona"), row["value"]["confidence"], row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 100_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 32915 samples


In [6]:
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

In [7]:
import pandas as pd

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [8]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [9]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

the file size of all the data is about 87 MB
the total number of rows in the file is 16052785
splitting into 18 files of about 5MB files with 891821 rows per file
saving rows 0 to 891820
confidence     0
value         70
Name: 2020-05-13 10:02:19-07:00, dtype: object
to a file named 2020-05-13T100219-0700_2020-08-03T034741-0700.parquet.gzip
2020-05-13 10:02:19-07:00
saving rows 891821 to 1783641
confidence     3
value         68
Name: 2020-08-03 03:47:46-07:00, dtype: object
to a file named 2020-08-03T034746-0700_2020-10-24T175335-0700.parquet.gzip
2020-08-03 03:47:46-07:00
saving rows 1783642 to 2675462
confidence     1
value         90
Name: 2020-10-24 17:53:50-07:00, dtype: object
to a file named 2020-10-24T175350-0700_2021-01-14T183521-0700.parquet.gzip
2020-10-24 17:53:50-07:00
saving rows 2675463 to 3567283
confidence     3
value         66
Name: 2021-01-14 18:35:31-07:00, dtype: object
to a file named 2021-01-14T183531-0700_2021-04-04T220303-0700.parquet.gzip
2021-01-14 18:35:31

In [10]:
# # Get the size of the file in bytes
# workingDataCompleteFile = "/home/chowder/Documents/workingData/fitbit/hr/fitbitHRdf.parquet.gzip"
# fitbitHRdf.to_parquet(workingDataCompleteFile,
#               compression='gzip') 

# file_size = os.path.getsize(workingDataCompleteFile)
# os.remove(workingDataCompleteFile)

In [11]:
# import math
# numFiles = math.ceil(file_size / (1024 * 1024 * 5))
# rows_per_file = int(len(fitbitHRdf)/numFiles)

# print(f"the file size of all the data is about {file_size // (1024 * 1024)} MB")
# print(f"the total number of rows in the file is {len(fitbitHRdf)}")
# print(f"splitting into {numFiles} number of files with {rows_per_file} rows per file")

# for fileNumber in range(numFiles + 1):
#     startRow = fileNumber * rows_per_file
#     if fileNumber == numFiles:
#         endRow = len(fitbitHRdf) - 1
#     else:
#         endRow = ((fileNumber + 1) * rows_per_file) - 1

#     print(f"saving rows {startRow} to {endRow}")
#     print(fitbitHRdf.iloc[startRow])

#     parquetName = fitbitHRdf.iloc[startRow].name.strftime('%Y-%m-%dT%H%M%S%z') +\
#                   "_" +\
#                   fitbitHRdf.iloc[endRow].name.strftime('%Y-%m-%dT%H%M%S%z') +\
#                   ".parquet.gzip"
#     print(f"to a file named {parquetName}")

#     print(pd.to_datetime(fitbitHRdf.iloc[startRow].name.strftime('%Y-%m-%dT%H%M%S%z')))

#     fitbitHRdf.iloc[startRow:endRow+1].to_parquet(fitbitHRWorkingDataPath + parquetName,
#               compression='gzip') 
